In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from rich.console import Console
from rich.table import Table

# 1) Load data (v3 - using new "real" files)
def load_data():
    """Loads the v3 long-format data."""
    try:
        vendors = pd.read_csv('vendors_real.csv')
        meds = pd.read_csv('meds_real.csv')
        inventory = pd.read_csv('vendor_inventory_real.csv')
        pharmacies = pd.read_csv('pharmacies_real.csv')
        return vendors, meds, inventory, pharmacies
    except FileNotFoundError:
        print("Error: v3 data files not found. Please run data_generator_v3.py first.")
        return [None] * 4

# 2) New Helper Function: Calculate a single vendor's plan (v3)
def get_vendor_plan_v3(vendor_id, vendor_details, vendor_stock, requirement, min_order_qty):
    """
    Calculates a single vendor's plan to meet a specific category requirement.
    Fills demand using the preference order.
    """
    demand_needed = requirement['demand']
    med_mix = []
    total_cost = 0
    total_qty = 0
    
    # Get all medicines for this category, sorted by preference
    substitute_meds = requirement['substitutes'].sort_values(by='preference')
    
    for _, med_row in substitute_meds.iterrows():
        med_id = med_row['medicine_id']
        
        if demand_needed <= 0:
            break
            
        # Check if this vendor has this specific med in stock
        stock_item = vendor_stock[vendor_stock['medicine_id'] == med_id]
        if stock_item.empty:
            continue # Vendor doesn't stock this med

        stock_available = stock_item.iloc[0]['stock']
        cost_per_unit = stock_item.iloc[0]['unit_cost']
        
        # How many can we take?
        qty_to_take = min(demand_needed, stock_available)
        
        if qty_to_take > 0:
            med_mix.append(f"{med_row['name']} ({qty_to_take})") # Use name
            total_cost += qty_to_take * cost_per_unit
            total_qty += qty_to_take
            demand_needed -= qty_to_take
            
    # Check if the plan is valid (met demand AND MOQ)
    if total_qty < requirement['demand'] or total_qty < min_order_qty:
        return None # This vendor can't fulfill the order
        
    delivery_time = vendor_details['base_delivery_time_hr']
    distance = vendor_details['distance_km'] # We'll add this
    
    # Final scoring for this *entire plan*
    # We prefer low cost, low time, low distance
    score = (1.0 / (total_cost + 1e-6)) * \
            (1.0 / (delivery_time + 1e-6)) * \
            (1.0 / (distance + 1e-6))

    return {
        'vendor_id': vendor_id,
        'vendor_name': vendor_details['name'],
        'total_cost': total_cost,
        'delivery_time_hr': delivery_time,
        'distance_km': distance,
        'total_quantity': total_qty,
        'medicine_mix': ", ".join(med_mix),
        'score': score
    }

# 3) Core Optimizer Function (Reworked for v3)
def run_category_optimization_v3(shopping_cart, pharmacy_id, min_order_qty=50, verbose=True):
    """
    Runs the optimization for a "shopping cart" of category requirements.
    Selects the ONE best vendor per category.
    """
    console = Console()
    
    vendors, meds, inventory, pharmacies = load_data()
    if vendors is None: return

    # --- Setup: Calculate distances to our pharmacy ---
    pharmacy = pharmacies[pharmacies['pharmacy_id'] == pharmacy_id].iloc[0]
    
    def distance_proxy(p_lat, p_lon, v_lat, v_lon):
        return (((p_lat - v_lat)**2 + (p_lon - v_lon)**2) ** 0.5) * 111
        
    vendors['distance_km'] = vendors.apply(
        lambda r: distance_proxy(pharmacy['lat'], pharmacy['lon'], r['lat'], r['lon']), axis=1
    )
    
    console.print(f"\n[bold cyan]Starting Optimization for Pharmacy {pharmacy_id}...[/bold cyan]")
    console.print(f"Minimum Order Quantity (MOQ): {min_order_qty} units\n")
    
    final_order_plan = []
    total_cost = 0
    
    # Create a copy of inventory to "consume" as we plan
    inventory_sim = inventory.copy()
    
    # --- Loop 1: By Requirement (Category) ---
    for req in shopping_cart:
        category = req['category']
        console.print(f"--- Analyzing Requirement: {req['demand']} units of [bold]{category}[/bold] ---")
        
        # Get all substitute medicines for this category
        substitutes = meds[meds['category'] == category].copy()
        req['substitutes'] = substitutes # Add to requirement for easy access
        
        pref_names = substitutes.sort_values(by='preference')['name'].tolist()
        console.print(f"Preference: {' > '.join(pref_names)}")
        
        vendor_plans = []
        # --- Loop 2: Evaluate All Vendors ---
        # Get all vendor inventory for this category's meds
        category_inventory = inventory_sim[inventory_sim['medicine_id'].isin(substitutes['medicine_id'])]
        
        for vendor_id in category_inventory['vendor_id'].unique():
            vendor_details = vendors[vendors['vendor_id'] == vendor_id].iloc[0]
            vendor_stock_for_category = category_inventory[category_inventory['vendor_id'] == vendor_id]
            
            plan = get_vendor_plan_v3(vendor_id, vendor_details, vendor_stock_for_category, req, min_order_qty)
            if plan:
                vendor_plans.append(plan)
                
        if not vendor_plans:
            console.print(f"[bold red]FAILED:[/bold red] No single vendor can fulfill the {category} requirement.\n")
            continue
            
        # We have our comparison table!
        plans_df = pd.DataFrame(vendor_plans).sort_values(by='score', ascending=False)
        
        # --- Display Comparison Table (as you requested) ---
        if verbose:
            table = Table(title=f"Vendor Comparison for [bold]{category}[/bold]")
            table.add_column("Vendor Name", style="cyan")
            table.add_column("Total Cost", style="green")
            table.add_column("Avg. Time (hr)", style="yellow")
            table.add_column("Dist (km)", style="magenta")
            table.add_column("Medicine Mix")
            
            for _, plan in plans_df.head(5).iterrows():
                table.add_row(
                    plan['vendor_name'],
                    f"₹{plan['total_cost']:.2f}",
                    f"{plan['delivery_time_hr']:.1f}",
                    f"{plan['distance_km']:.1f}",
                    plan['medicine_mix']
                )
            console.print(table)
            
        # --- Select the BEST plan (top of the sorted list) ---
        best_plan = plans_df.iloc[0].to_dict()
        final_order_plan.append(best_plan)
        total_cost += best_plan['total_cost']
        
        console.print(f"[bold green]Decision:[/bold green] Ordering from [bold]{best_plan['vendor_name']}[/bold].")
        console.print(f"Cost: ₹{best_plan['total_cost']:.2f}, Mix: {best_plan['medicine_mix']}\n")
        
        # --- Update Vendor Stock for next category (CRITICAL) ---
        # This simulates the order being placed, so the next
        # category can't use the same stock.
        for med_mix_str in best_plan['medicine_mix'].split(', '):
            # Parse 'Calpol 500mg (100)'
            med_name = med_mix_str.split(' (')[0]
            qty_str = med_mix_str.split(' (')[1].strip(')')
            qty = int(qty_str)
            
            # Find the med_id from the name
            med_id = meds[meds['name'] == med_name].iloc[0]['medicine_id']
            
            # Find the row in the main inventory dataframe and subtract the stock
            idx_to_update = inventory_sim[
                (inventory_sim['vendor_id'] == best_plan['vendor_id']) &
                (inventory_sim['medicine_id'] == med_id)
            ].index[0]
            
            inventory_sim.loc[idx_to_update, 'stock'] -= qty

    # --- 4) Final Summary ---
    console.print("\n--- [bold]Final Consolidated Order Plan[/bold] ---")
    
    summary_table = Table(title="Final Order")
    summary_table.add_column("Category")
    summary_table.add_column("Vendor")
    summary_table.add_column("Quantity")
    summary_table.add_column("Medicine Mix")
    summary_table.add_column("Cost")
    
    # Need to re-map category
    for plan, req in zip(final_order_plan, shopping_cart):
        summary_table.add_row(
            req['category'],
            plan['vendor_name'],
            str(plan['total_quantity']),
            plan['medicine_mix'],
            f"₹{plan['total_cost']:.2f}"
        )
    
    console.print(summary_table)
    console.print(f"[bold]Total Estimated Cost: ₹{total_cost:.2f}[/bold]")
    return final_order_plan

# --- Example Usage ---
if __name__ == "__main__":
    
    # Define the pharmacy's total shopping needs
    shopping_cart = [
        {
            'category': 'Fever', 
            'demand': 120, 
            # Note: Preference is now read from the 'meds_real.csv'
        },
        {
            'category': 'Stomach', 
            'demand': 80, 
        },
        {
            'category': 'Toothache', 
            'demand': 50, 
        }
    ]
    
    # Set the minimum order quantity for any single vendor order
    min_order_qty = 50
    
    # Pick a pharmacy to run the simulation for (e.g., pharmacy 101)
    pharmacy_id = 101
    
    # Run the new optimizer
    # (Requires 'rich' library: pip install rich)
    plan = run_category_optimization_v3(shopping_cart, pharmacy_id, min_order_qty, verbose=True)

